# Make average and weighted word embedding for each document
- Load df
- Load pretrained word vector model
    - I used Glove, but FastText may be better
- get centroids with glove vecs (300)
- get glove x tfidf weighted vecs (300)

In [1]:
import pickle
import pandas as pd, numpy as np

import gensim

In [2]:
### load df

DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df_cleaned.pkl'

with open(DATA_PATH + DF_NAME, 'rb') as f:
    df = pickle.load(f)

In [3]:
### Load word embeddings model
# Here I use the Stanford GloVe model

from gensim.models import KeyedVectors
MODEL_PATH = './models/'
model_filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(MODEL_PATH + model_filename, binary=False)

In [4]:
def get_avg_word_embedding(sent: str, model):
    errors = 0
    sent_sum = np.zeros(100)
    for word in sent:
        try:
            vec = model[word]
            sent_sum += vec
        except:
            errors +=1
            pass

    sent_avg = sent_sum / len(sent_sum)
    return sent_avg

In [5]:
df['glove_avg'] = df['article_text_cleaned'].apply(lambda x: get_avg_word_embedding(x, model))

In [6]:
filename = 'concatenated_df_cleaned_glove.pkl'
with open(DATA_PATH + filename, 'wb') as f:
    pickle.dump(df, f)

## tf-idf weighting

#### gensim tfidf giving problems

In [ ]:
# from gensim.models import TfidfModel

# MODEL_PATH = './models/'
# tfidf_name = 'tfidf.model'
# tfidf = TfidfModel.load(MODEL_PATH + tfidf_name)

# ### load dictionary
# DATA_PATH = './data/cleaned/'
# articles_dict_filename = 'articles_dict.pkl'

# with open(DATA_PATH + articles_dict_filename, 'rb') as f:
#     articles_dict = pickle.load(f)

    
# ### load corpus
# articles_corpus_filename = 'articles_corpus.pkl'

# with open(DATA_PATH + articles_corpus_filename, 'rb') as f:
#     articles_corpus = pickle.load(f)

# # fit to the corpus
# corpus_tfidf = tfidf[articles_corpus]

# print('type of tfidf model: {}'.format(type(tfidf)))
# print('type of corpus_tfidf: {}'.format(type(corpus_tfidf)))

### tfidf from sklearn
#### weighting word vectors with tf-idf takes too long... (12+ hours)

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer


df['article_text_cleaned_string'] = df['article_text_cleaned'].apply(lambda x: ' '.join(x))


# !pip install tdqm
from tqdm import tqdm

tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(df['article_text_cleaned_string'])
tfidf_feat = tf_idf_vect.get_feature_names()

import time
t0 = time.time()

#Applying TF-IDF scores to the model vectors
tfidf_sent_vectors = [] # the tfidf-w2v for each sentence/review is stored in this list
row=0
errors=0


for sent in tqdm(df['article_text_cleaned'].tolist()): # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = model[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf [row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))
    tfidf_sent_vectors.append(sent_vec)
    row += 1
print('errors noted: '+str(errors))

seconds = time.time()-t0
minutes = seconds/60
hours = minutes / 60
hours_min = minutes % 60

print('{} hours and {} minutes'.format(hours, hours_min))

len(tfidf_sent_vectors)
type(tfidf_sent_vectors)

DATA_PATH = './data/cleaned/'
filename = 'tfidf_sent_vectors'
with open(DATA_PATH + filename, 'wb') as f:
    pickle.dump(tfidf_sent_vectors, f)

### parking lot

In [ ]:
df['article_text_cleaned'][0]

In [ ]:
row = 0
word = 'vaccine'
print(word)
print('tfidf_feat.index(' + word + '): {}'.format(tfidf_feat.index(word)))
tfidf_sklearn[[row, tfidf_feat.index(word)]]

In [ ]:
from tqdm import tqdm

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
errors=0
for sent in tqdm(tokens): # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum = 0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = model[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = tfidf_sklearn [row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))
tfidf_sent_vectors.append(sent_vec)
    row += 1
print('errors noted: '+str(errors))